# Mining for the Future: Digital Mineral Exploration for a Sustainable Tomorrow

This notebook contains the code to predict the presence of porphyry copper deposits in a polygon uploaded by the user. This polygon is assumed to be a geoJSON. The notebook prepares the polygon for prediction and feeds it to a function that generates

## Package installs and imports

In [ ]:
pip install geemap

In [ ]:
pip install rasterio

In [ ]:
pip install mapclassify

Use the command `!cp -r source_folder destination_folder` to load the custom modules `preprocessing`, `band_engineering`, `array_processing`, `prediction_model`, and `pred_from_polygon` into colab session storage.

Note that the weights_path variable in the `prediction_model` module may need to be adjusted before `pred_from_polygon` is imported.

In [ ]:
import geemap
import json
import mapclassify
import pred_from_polygon
import ee

ee.Authenticate()
ee.Initialize()

## Load the input polygon and convert to ee.Geometry object

In [ ]:
polygon_file_path = "" # File path to polygon stored as a geoJSON
with open(polygon_file_path, 'r') as file:
  polygon_json = json.load(file)

In [ ]:
polygon_ee = geemap.geojson_to_ee(polygon_json)

## Predict the occurrence of Porphyry Copper Deposits inside the polygon

The `PCD_prediction` function implements a multi-step process to generate predictions of PCD occurrence inside the polygon.


1.   Calculate the minimum and maximum coordinates of the polygon (`pred_from_polygon.poly_min_max_coords`).
2.   Generate a grid of points spaced 3 km apart within those limits. These points will be used to gather and process the satellite imagery that forms the basis of prediction (`pred_from_polygon.create_input_grid_points`).
3. Generate a second grid of points, also spaced 3 km apart, but offset from the first grid by 1.5 km. These points will be used to calculate the average predictions from overlapping sets of imagery (`pred_from_polygon.create_result_grid_points`).
4. Clip both grids to keep only points inside the polygon (`pred_from_polygon.clip_grids_points`).
5.  Convert the first grid from an ee.FeatureCollection object to a list of ee.Geometry objects (`pred_from_polygon.convert_fc_to_geometry_list`).
6. Iterate over that list, performing the following operations for each point:
    * Get preprocessed and band-engineered ASTER imagery for a 7 x 7 km box centered on the point, generate a download link for that image, download the image into memory and convert it to a numpy array (`pred_from_polygon.gee_image_to_numpy`).
    * Process the array to make it suitable for prediction and pass it to the model for prediction (`pred_from_polygon.model_prediction`).
    * Create a 6 x 6 km box representing the footprint of the image passed to the model.
    * Convert the box to an ee.Feature object and set the model prediction as a property on that object.
    * Append the ee.Feature object to a list called `res_features_pred`.
7. Convert the second grid from an ee.FeatureCollection object to a list of ee.Geometry objects (`pred_from_polygon.convert_fc_to_geometry_list`)
8. Iterate over that list, performing the following operations for each point:
    * Create an empty list to store predictions from boxes that overlap the point called `predictions`.
    * Iterate over all of boxes with predictions in the `res_features_pred` list and, if the box contains the point, append the box's prediction to the `predictions` list.
    * Calculate the average of all the predictions in the `predictions` list
    * Create a 3 x 3 km box centered on the point representing the area of overlapping predictions included in the `predictions` list (`pred_from_polygon.create_box_avg_predictions`)
    * Convert that box to an ee.Feature object and set the average prediction as a property on that object.
    * Append the ee.Feature object to a list called `res_poly_list`.
9. Convert `res_poly_list` to an ee.FeatureCollection and return it.


In [ ]:
predictions_ee = pred_from_polygon.PCD_prediction(polygon_ee)

## Visualize the results

To visualize the results on a map, the ee.FeatureCollection is converted to a Geopandas DataFrame and the coordinate reference system (crs) is defined. The `geemap.classify` function classifies the continuous data in the `avg_prediction` column for display purposes.

In [ ]:
predictions_gpd = geemap.ee_to_geopandas(predictions_ee)
predictions_gpd = predictions_gpd.set_crs('EPSG:4326')
data, legend = geemap.classify(predictions_gpd, 'avg_prediction')

Instantiating a `geemap.Map` object and displaying it allows one to select a basemap before adding layers.

In [ ]:
map = geemap.Map()
map

The `.add_data` method adds a layer to the map that displays data according to a user-specified scheme. Setting the `scheme` to `'UserDefined'` gives the user precise control over the classification parameters that determine how different values are represented. This is achieved by manipulating the values in the the dictionary passed to `classification_kwds`.

In [ ]:
map.add_data(data,
             column = 'avg_prediction',
             scheme = 'UserDefined',
             cmap = 'OrRd',
             legend_title = 'Average Prediction',
             k = 5,
             info_mode = 'on_click',
             classification_kwds = {'bins': [0.2, 0.4, 0.6, 0.8, 1]})
map.centerObject(polygon_ee)

In [ ]:
map